# 03_fairness_and_xai.ipynb
## Fairness Evaluation & Explainable AI
We evaluate the trained model for gender bias and use SHAP and LIME for explainability.

## Fairness Metrics
Calculating accuracy and selection rate by gender.

## SHAP Explainability
Visualizing which words influence model predictions.

## LIME Explainability
Explaining a single prediction with LIME.


In [ ]:
# 03_fairness_and_xai.ipynb

import pandas as pd
import joblib
import shap
from fairlearn.metrics import MetricFrame, selection_rate, demographic_parity_difference
import matplotlib.pyplot as plt
from lime.lime_text import LimeTextExplainer

# Load processed data
df = pd.read_csv("data/processed/processed_data.csv")
X_text = df["cleaned_resume"]
y = df["label"]
sensitive = df["gender"]

# Load model & vectorizer
model = joblib.load("artifacts/logreg_model.pkl")
tfidf = joblib.load("artifacts/tfidf_vectorizer.pkl")
X_vect = tfidf.transform(X_text)

# Fairness metrics
y_pred = model.predict(X_vect)
metrics = {"accuracy": accuracy_score, "selection_rate": selection_rate}
mf = MetricFrame(metrics=metrics, y_true=y, y_pred=y_pred, sensitive_features=sensitive)
print("Group-wise metrics:")
print(mf.by_group)

dp_diff = demographic_parity_difference(y, y_pred, sensitive_features=sensitive)
print("\nDemographic Parity Difference:", dp_diff)

# SHAP
explainer = shap.LinearExplainer(model, X_vect)
shap_values = explainer.shap_values(X_vect)
shap.summary_plot(shap_values, X_vect, feature_names=tfidf.get_feature_names_out(), show=True)

# LIME example
explainer_lime = LimeTextExplainer(class_names=["Rejected","Selected"])
exp = explainer_lime.explain_instance(X_text.iloc[0],
                                      lambda x: model.predict_proba(tfidf.transform(x)),
                                      num_features=10)
exp.show_in_notebook()
